In [1]:
import os
import simulation
import pynbody
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import ipywidgets
import tqdm
from simulation.sfh_in_box import sfh, plot_sfh, plot_binned_sfh
from data_pd.dump_features import dump_features
from astropy import constants as c
from astropy.table import Table

In [2]:
SIMPATH = '/home/michele/sim/MySimulations/ng'

In [3]:
SIM = 'mb.71002'
TRAJ = 'p100_a800_r600'

In [4]:
sim_path = os.path.join(SIMPATH, "{}_{}".format(SIM, TRAJ), "out")
NTH = 10
sim = simulation.Simulation(sim_path, snap_indexes=slice(None, None, NTH))

loading simulation: /home/michele/sim/MySimulations/ng/mb.71002_p100_a800_r600/out
Found parameter file
Found compiler file
Found 563 snapshots
Taking 57 snapshots (slice(None, None, 10))
Loading cosmological parameters
Found trace file


In [5]:
# load_cache = True
# radius = 5
# outname = 'data_pd/{}_{}_s{}_n{}.fits'.format(SIM, TRAJ, radius, NTH)
# if load_cache:
#     tbl = Table.read(outname)
# else:
#     dump_features(sim, outname, radius=radius)
#     tbl = Table.read(outname)

In [6]:
from simulation.derived import mgfe, feh
snap = sim[50]

In [7]:
snap.s['mass'].in_units('Msol').mean()

SimArray(8646.799, dtype=float32, 'Msol')

In [8]:
snap

<SimSnap "/home/michele/sim/MySimulations/ng/mb.71002_p100_a800_r600/out/snapshot_0501" len=17625>

In [9]:
snap.properties

{'time': Unit("1.34e+01 s kpc km**-1"),
 'h': 0.7,
 'omegaL0': 0.72,
 'omegaM0': 0.28}

In [10]:
pynbody.config['sph']['smooth-particles'] = 50

In [11]:
pynbody.analysis.halo.transformation.GenericTranslation?

Init signature: pynbody.analysis.halo.transformation.GenericTranslation(f, arname, shift)
Docstring:      <no docstring>
File:           ~/Software/miniconda3/lib/python3.6/site-packages/pynbody/transformation.py
Type:           type
Subclasses:     


In [12]:
sl = ipywidgets.IntSlider(min=0, max=len(sim)-1, value=0)
delta = 30
def k(i): 
    snap = sim[i]
    print(len(snap.g))
    
#     pynbody.analysis.halo.center(snap.s)
    with pynbody.transformation.translate(snap, shift=[-30] * 3):
        pynbody.plot.image(snap.g, qty='rho', av_z=True, width=30, log=True)#, vmin=0, vmax=0.5)#, vmin=-10.5, vmax=-1);

In [13]:
w = ipywidgets.interactive(k, i=sl)
w.children[-1].layout.height = '350px'
w

interactive(children=(IntSlider(value=0, description='i', max=56), Output(layout=Layout(height='350px'))), _do…

In [14]:
sim.properties.get('boxsize'), snap.properties.get('boxsize')

(None, None)

In [17]:
sl = ipywidgets.IntSlider(min=0, max=len(sim)-1, value=0)
def k(i): 
    snap = sim[i]
    pynbody.analysis.halo.center(snap.s)
    pynbody.plot.image(snap.s, qty='feh', width=4, log=False, vmin=-2.5, vmax=0);

In [21]:
w = ipywidgets.interactive(k, i=sl)
w.children[-1].layout.height = '280px'
w

interactive(children=(IntSlider(value=0, description='i', max=56), Output(layout=Layout(height='280px'))), _do…

## Deriving functions machinery

In [64]:
fam=snap.g._unifamily

In [65]:
snap._family_derived_array_names[fam]

['mu', 'cs']

In [66]:
snap._derived_array_names

[]

In [68]:
snap.is_derived_array('p')

False

In [49]:
snap._family_derived_array_names[fam]

['mgfe']

In [70]:
snap._find_deriving_function('mgfe')

<function simulation.derived.mgfe(snap)>

In [72]:
snap._dependency_tracker.calculating

<bound method DependencyTracker.calculating of <pynbody.dependencytracker.DependencyTracker object at 0x7f7599ed1358>>

In [89]:
pynbody.family.get_family('g')

<Family gas>

In [104]:
snap._unifamily is pynbody.family.star

False